# Lab | Langchain Evaluation

## Intro

Pick different sets of data and re-run this notebook. The point is for you to understand all steps involve and the many different ways one can and should evaluate LLM applications.

What did you learn? - Let's discuss that in class

## LangChain: Evaluation

### Outline:

* Example generation
* Manual evaluation (and debuging)
* LLM-assisted evaluation

In [1]:
from dotenv import load_dotenv, find_dotenv
import os
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY') 

### Example 1

#### Create our QandA application

In [2]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI
from langchain.llms import OpenAI
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.document_loaders import CSVLoader, TextLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.chains import LLMChain


In [3]:
import os
file = './OutdoorClothingCatalog_1000.csv'
if not os.path.exists(file):
    raise FileNotFoundError(f"File not found: {file}")

In [4]:
import pandas as pd

file = './OutdoorClothingCatalog_1000.csv'
try:
    df = pd.read_csv(file)
    print(df.head())
except Exception as e:
    print(f"Error: {e}")

   Unnamed: 0                                               name  \
0           0                           Women's Campside Oxfords   
1           1           Recycled Waterhog Dog Mat, Chevron Weave   
2           2  Infant and Toddler Girls' Coastal Chill Swimsu...   
3           3         Refresh Swimwear, V-Neck Tankini Contrasts   
4           4                             EcoFlex 3L Storm Pants   

                                         description  
0  This ultracomfortable lace-to-toe Oxford boast...  
1  Protect your floors from spills and splashing ...  
2  She'll love the bright colors, ruffles and exc...  
3  Whether you're going for a swim or heading out...  
4  Our new TEK O2 technology makes our four-seaso...  


In [5]:
from langchain.document_loaders import CSVLoader

file = './OutdoorClothingCatalog_1000.csv'
try:
    loader = CSVLoader(file_path=file, encoding='ISO-8859-1')  # Specify encoding
    data = loader.load()
    print(data[:5])  # Print the first 5 entries to verify
except Exception as e:
    print(f"Error: {e}")

[Document(metadata={'source': './OutdoorClothingCatalog_1000.csv', 'row': 0}, page_content=": 0\nname: Women's Campside Oxfords\ndescription: This ultracomfortable lace-to-toe Oxford boasts a super-soft canvas, thick cushioning, and quality construction for a broken-in feel from the first time you put them on. \n\nSize & Fit: Order regular shoe size. For half sizes not offered, order up to next whole size. \n\nSpecs: Approx. weight: 1 lb.1 oz. per pair. \n\nConstruction: Soft canvas material for a broken-in feel and look. Comfortable EVA innersole with Cleansport NXTÂ® antimicrobial odor control. Vintage hunt, fish and camping motif on innersole. Moderate arch contour of innersole. EVA foam midsole for cushioning and support. Chain-tread-inspired molded rubber outsole with modified chain-tread pattern. Imported. \n\nQuestions? Please contact us for any inquiries."), Document(metadata={'source': './OutdoorClothingCatalog_1000.csv', 'row': 1}, page_content=': 1\nname: Recycled Waterhog D

In [6]:
# !pip install --upgrade --force-reinstall sentence-transformers

In [7]:
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2", model_kwargs = {'device': 'cpu'})
).from_loaders([loader])

c:\Users\alvar\AppData\Local\Programs\Python\Python313\Lib\site-packages\pydantic\_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [8]:
llm = ChatOpenAI(temperature = 0.0)
qa = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=index.vectorstore.as_retriever(), 
    verbose=True,
    chain_type_kwargs = {
        "document_separator": "<<<<>>>>>"
    }
)

#### Coming up with test datapoints

In [9]:
data[10]

Document(metadata={'source': './OutdoorClothingCatalog_1000.csv', 'row': 10}, page_content=": 10\nname: Cozy Comfort Pullover Set, Stripe\ndescription: Perfect for lounging, this striped knit set lives up to its name. We used ultrasoft fabric and an easy design that's as comfortable at bedtime as it is when we have to make a quick run out.\n\nSize & Fit\n- Pants are Favorite Fit: Sits lower on the waist.\n- Relaxed Fit: Our most generous fit sits farthest from the body.\n\nFabric & Care\n- In the softest blend of 63% polyester, 35% rayon and 2% spandex.\n\nAdditional Features\n- Relaxed fit top with raglan sleeves and rounded hem.\n- Pull-on pants have a wide elastic waistband and drawstring, side pockets and a modern slim leg.\n\nImported.")

In [10]:
data[11]

Document(metadata={'source': './OutdoorClothingCatalog_1000.csv', 'row': 11}, page_content=': 11\nname: Ultra-Lofty 850 Stretch Down Hooded Jacket\ndescription: This technical stretch down jacket from our DownTek collection is sure to keep you warm and comfortable with its full-stretch construction providing exceptional range of motion. With a slightly fitted style that falls at the hip and best with a midweight layer, this jacket is suitable for light activity up to 20Â° and moderate activity up to -30Â°. The soft and durable 100% polyester shell offers complete windproof protection and is insulated with warm, lofty goose down. Other features include welded baffles for a no-stitch construction and excellent stretch, an adjustable hood, an interior media port and mesh stash pocket and a hem drawcord. Machine wash and dry. Imported.')

#### Hard-coded examples

In [11]:
from langchain.prompts import PromptTemplate

In [12]:
from langchain.prompts import PromptTemplate
from langchain.schema import BaseOutputParser
from pydantic import BaseModel, Field

examples = [
    {
        "query": "Do the Cozy Comfort Pullover Set\
        have side pockets?",
        "answer": "Yes"
    },
    {
        "query": "What collection is the Ultra-Lofty \
        850 Stretch Down Hooded Jacket from?",
        "answer": "The DownTek collection"
    }
]

# Define the prompt template
prompt_template = PromptTemplate(
    input_variables=["query"],
    template="Examples:\n"
             "1. Query: Do the Cozy Comfort Pullover Set have side pockets?\n"
             "   Answer: Yes\n"
             "2. Query: What collection is the Ultra-Lofty 850 Stretch Down Hooded Jacket from?\n"
             "   Answer: The DownTek collection\n"
             "Query: {query}\n"
             "Answer:"
)

# Define the output model
class Answer(BaseModel):
    answer: str = Field(description="The answer to the query")

# Create the output parser
class AnswerOutputParser(BaseOutputParser):
    def parse(self, text: str) -> Answer:
        # Split the response to get the answer
        answer = text.strip().split("Answer:")[-1].strip()
        return Answer(answer=answer)

# Initialize the LLM
# llm = OpenAI()
llm = ChatOpenAI()

# Create the LLMChain
llm_chain = LLMChain(
    llm=llm,
    prompt=prompt_template,
    output_parser=AnswerOutputParser()
)

# Example query
query = "Is the Cozy Comfort Pullover Set available in different colors?"

# Run the chain
result = llm_chain.run({"query": query})

# Print the result
print(result)


C:\Users\alvar\AppData\Local\Temp\ipykernel_17568\1856079041.py:46: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(
C:\Users\alvar\AppData\Local\Temp\ipykernel_17568\1856079041.py:56: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = llm_chain.run({"query": query})


answer='Yes, the Cozy Comfort Pullover Set is available in multiple colors such as grey, navy blue, and burgundy.'


#### LLM-Generated examples

In [13]:
from langchain.evaluation.qa import QAGenerateChain

In [14]:
example_gen_chain = QAGenerateChain.from_llm(ChatOpenAI())

In [15]:
llm_chain = LLMChain(llm=llm, prompt=prompt_template)

In [16]:
new_examples = example_gen_chain.apply_and_parse(
    [{"doc": t} for t in data[:5]]
)

c:\Users\alvar\AppData\Local\Programs\Python\Python313\Lib\site-packages\langchain\chains\llm.py:369: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


In [17]:
new_examples[0]

{'qa_pairs': {'query': "What is the recommended sizing information for the Women's Campside Oxfords?",
  'answer': "The recommended sizing information for the Women's Campside Oxfords is to order your regular shoe size. For half sizes not offered, it is recommended to order up to the next whole size."}}

In [18]:
data[0]

Document(metadata={'source': './OutdoorClothingCatalog_1000.csv', 'row': 0}, page_content=": 0\nname: Women's Campside Oxfords\ndescription: This ultracomfortable lace-to-toe Oxford boasts a super-soft canvas, thick cushioning, and quality construction for a broken-in feel from the first time you put them on. \n\nSize & Fit: Order regular shoe size. For half sizes not offered, order up to next whole size. \n\nSpecs: Approx. weight: 1 lb.1 oz. per pair. \n\nConstruction: Soft canvas material for a broken-in feel and look. Comfortable EVA innersole with Cleansport NXTÂ® antimicrobial odor control. Vintage hunt, fish and camping motif on innersole. Moderate arch contour of innersole. EVA foam midsole for cushioning and support. Chain-tread-inspired molded rubber outsole with modified chain-tread pattern. Imported. \n\nQuestions? Please contact us for any inquiries.")

In [19]:
d_flattened = [data['qa_pairs'] for data in new_examples]
d_flattened

[{'query': "What is the recommended sizing information for the Women's Campside Oxfords?",
  'answer': "The recommended sizing information for the Women's Campside Oxfords is to order your regular shoe size. For half sizes not offered, it is recommended to order up to the next whole size."},
 {'query': 'What are the dimensions of the small and medium Recycled Waterhog Dog Mat in the document?',
  'answer': 'The small Recycled Waterhog Dog Mat has dimensions of 18" x 28", while the medium size has dimensions of 22.5" x 34.5".'},
 {'query': "What are some features of the Infant and Toddler Girls' Coastal Chill Swimsuit, Two-Piece mentioned in the document?",
  'answer': 'The features of the swimsuit include bright colors, ruffles, exclusive whimsical prints, four-way-stretch and chlorine-resistant fabric, UPF 50+ rated fabric for sun protection, crossover no-slip straps, fully lined bottom for secure fit and coverage, and it is recommended to be machine washed and line dried for best res

#### Combine examples

In [20]:
# examples += new_example
examples += d_flattened

In [21]:
examples[0]

{'query': 'Do the Cozy Comfort Pullover Set        have side pockets?',
 'answer': 'Yes'}

In [22]:
qa.invoke(examples[0]["query"])



> Entering new RetrievalQA chain...

> Finished chain.


{'query': 'Do the Cozy Comfort Pullover Set        have side pockets?',
 'result': 'The Cozy Comfort Pullover Set does not mention having side pockets in the provided context.'}

### Manual Evaluation - Fun part

In [23]:
import langchain
langchain.debug = True

In [24]:
qa.invoke(examples[0]["query"])

[chain/start] [chain:RetrievalQA] Entering Chain run with input:
{
  "query": "Do the Cozy Comfort Pullover Set        have side pockets?"
}
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "question": "Do the Cozy Comfort Pullover Set        have side pockets?",
  "context": "All pockets have sturdy pocket bags and offer plenty of room for a wallet, cell phone and more.\n\nGusseted crotch for ease of movement.\n\nImported.<<<<>>>>>: 73\nname: Cozy Cuddles Knit Pullover Set\ndescription: Perfect for lounging, this knit set lives up to its name. We used ultrasoft fabric and an easy design that's as comfortable at bedtime as it is when we have to make a quick run out. \n\nSize & Fit \nPants are Favorite Fit: Sits lower on the waist. \nRelaxed Fit: Our most generous fit sits farthest from the body. \n\nFabric & Care \nIn the

{'query': 'Do the Cozy Comfort Pullover Set        have side pockets?',
 'result': 'The Cozy Comfort Pullover Set does not mention having side pockets in the provided context.'}

In [25]:
# Turn off the debug mode
langchain.debug = False

### LLM assisted evaluation

In [26]:
examples += d_flattened

In [27]:
examples

[{'query': 'Do the Cozy Comfort Pullover Set        have side pockets?',
  'answer': 'Yes'},
 {'query': 'What collection is the Ultra-Lofty         850 Stretch Down Hooded Jacket from?',
  'answer': 'The DownTek collection'},
 {'query': "What is the recommended sizing information for the Women's Campside Oxfords?",
  'answer': "The recommended sizing information for the Women's Campside Oxfords is to order your regular shoe size. For half sizes not offered, it is recommended to order up to the next whole size."},
 {'query': 'What are the dimensions of the small and medium Recycled Waterhog Dog Mat in the document?',
  'answer': 'The small Recycled Waterhog Dog Mat has dimensions of 18" x 28", while the medium size has dimensions of 22.5" x 34.5".'},
 {'query': "What are some features of the Infant and Toddler Girls' Coastal Chill Swimsuit, Two-Piece mentioned in the document?",
  'answer': 'The features of the swimsuit include bright colors, ruffles, exclusive whimsical prints, four-wa

In [28]:
predictions = qa.batch(examples)



> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.


In [29]:
predictions

[{'query': 'Do the Cozy Comfort Pullover Set        have side pockets?',
  'answer': 'Yes',
  'result': 'The Cozy Comfort Pullover Set does not mention having side pockets in the provided context.'},
 {'query': 'What collection is the Ultra-Lofty         850 Stretch Down Hooded Jacket from?',
  'answer': 'The DownTek collection',
  'result': 'The Ultra-Lofty 850 Stretch Down Hooded Jacket is from the DownTek collection.'},
 {'query': "What is the recommended sizing information for the Women's Campside Oxfords?",
  'answer': "The recommended sizing information for the Women's Campside Oxfords is to order your regular shoe size. For half sizes not offered, it is recommended to order up to the next whole size.",
  'result': 'I\'m sorry, but there is no specific product named "Women\'s Campside Oxfords" in the provided context. If you have more details or a different name for the product, I can try to assist you further.'},
 {'query': 'What are the dimensions of the small and medium Recycl

In [30]:
from langchain.evaluation.qa import QAEvalChain

In [31]:
llm = ChatOpenAI(temperature=0)
eval_chain = QAEvalChain.from_llm(llm)

In [32]:
graded_outputs = eval_chain.evaluate(examples, predictions)

In [33]:
graded_outputs

[{'results': 'INCORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'INCORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'CORRECT'}]

In [34]:
for i, eg in enumerate(examples):
    print(f"Example {i}:")
    print("Question: " + predictions[i]['query'])
    print("Real Answer: " + predictions[i]['answer'])
    print("Predicted Answer: " + predictions[i]['result'])
    # print("Predicted Grade: " + graded_outputs[i]['text'])
    print()

Example 0:
Question: Do the Cozy Comfort Pullover Set        have side pockets?
Real Answer: Yes
Predicted Answer: The Cozy Comfort Pullover Set does not mention having side pockets in the provided context.

Example 1:
Question: What collection is the Ultra-Lofty         850 Stretch Down Hooded Jacket from?
Real Answer: The DownTek collection
Predicted Answer: The Ultra-Lofty 850 Stretch Down Hooded Jacket is from the DownTek collection.

Example 2:
Question: What is the recommended sizing information for the Women's Campside Oxfords?
Real Answer: The recommended sizing information for the Women's Campside Oxfords is to order your regular shoe size. For half sizes not offered, it is recommended to order up to the next whole size.
Predicted Answer: I'm sorry, but there is no specific product named "Women's Campside Oxfords" in the provided context. If you have more details or a different name for the product, I can try to assist you further.

Example 3:
Question: What are the dimensions

### Example 2
One can also easily evaluate your QA chains with the metrics offered in ragas

In [42]:
from langchain_huggingface import HuggingFaceEmbeddings
loader = CSVLoader(file_path='./nyc_text.txt', encoding='ISO-8859-1')
index = VectorstoreIndexCreator(embedding=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2", model_kwargs={'device': 'cpu'})).from_loaders([loader])


llm = ChatOpenAI(temperature= 0)
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=index.vectorstore.as_retriever(),
    return_source_documents=True,
)

c:\Users\alvar\AppData\Local\Programs\Python\Python313\Lib\site-packages\langchain\indexes\vectorstore.py:171: UserWarning: Using InMemoryVectorStore as the default vectorstore.This memory store won't persist data. You should explicitlyspecify a vectorstore when using VectorstoreIndexCreator
  warnings.warn(


In [43]:
# testing it out

question = "How did New York City get its name?"
result = qa_chain.invoke({"query": question})
result["result"]

'New York City was named in honor of the Duke of York, who would become King James II of England. King Charles II appointed the Duke as the proprietor of the former territory of New Netherland, including the city of New Amsterdam, when England seized it from Dutch control in 1664.'

In [44]:
result

{'query': 'How did New York City get its name?',
 'result': 'New York City was named in honor of the Duke of York, who would become King James II of England. King Charles II appointed the Duke as the proprietor of the former territory of New Netherland, including the city of New Amsterdam, when England seized it from Dutch control in 1664.',
 'source_documents': [Document(id='0318c37c-9d34-4f8a-8724-4ac9555e91e5', metadata={'source': './nyc_text.txt', 'row': 0}, page_content="804: the Duke of York. The city was regained by the Dutch in July 1673 and was renamed New Orange for one year and three months; the city has been continuously named New York since November 1674. New York City was the capital of the United States from 1785 until 1790\n190 distributed over 300.46 square miles (778.2 km2): and has been the largest U.S. city since 1790. The Statue of Liberty greeted millions of immigrants as they came to the U.S. by ship in the late 19th and early 20th centuries\nNew York City is the

Now in order to evaluate the qa system we generated a few relevant questions. We've generated a few question for you but feel free to add any you want.

In [45]:
eval_questions = [
    "What is the population of New York City as of 2020?",
    "Which borough of New York City has the highest population?",
    "What is the economic significance of New York City?",
    "How did New York City get its name?",
    "What is the significance of the Statue of Liberty in New York City?",
]

eval_answers = [
    "8,804,190",
    "Brooklyn",
    "New York City's economic significance is vast, as it serves as the global financial capital, housing Wall Street and major financial institutions. Its diverse economy spans technology, media, healthcare, education, and more, making it resilient to economic fluctuations. NYC is a hub for international business, attracting global companies, and boasts a large, skilled labor force. Its real estate market, tourism, cultural industries, and educational institutions further fuel its economic prowess. The city's transportation network and global influence amplify its impact on the world stage, solidifying its status as a vital economic player and cultural epicenter.",
    "New York City got its name when it came under British control in 1664. King Charles II of England granted the lands to his brother, the Duke of York, who named the city New York in his own honor.",
    "The Statue of Liberty in New York City holds great significance as a symbol of the United States and its ideals of liberty and peace. It greeted millions of immigrants who arrived in the U.S. by ship in the late 19th and early 20th centuries, representing hope and freedom for those seeking a better life. It has since become an iconic landmark and a global symbol of cultural diversity and freedom.",
]

examples = [
    {"query": q, "ground_truths": [eval_answers[i]]}
    for i, q in enumerate(eval_questions)
]

In [46]:
examples

[{'query': 'What is the population of New York City as of 2020?',
  'ground_truths': ['8,804,190']},
 {'query': 'Which borough of New York City has the highest population?',
  'ground_truths': ['Brooklyn']},
 {'query': 'What is the economic significance of New York City?',
  'ground_truths': ["New York City's economic significance is vast, as it serves as the global financial capital, housing Wall Street and major financial institutions. Its diverse economy spans technology, media, healthcare, education, and more, making it resilient to economic fluctuations. NYC is a hub for international business, attracting global companies, and boasts a large, skilled labor force. Its real estate market, tourism, cultural industries, and educational institutions further fuel its economic prowess. The city's transportation network and global influence amplify its impact on the world stage, solidifying its status as a vital economic player and cultural epicenter."]},
 {'query': 'How did New York City

#### Introducing RagasEvaluatorChain

`RagasEvaluatorChain` creates a wrapper around the metrics ragas provides (documented [here](https://github.com/explodinggradients/ragas/blob/main/docs/metrics.md)), making it easier to run these evaluation with langchain and langsmith.

The evaluator chain has the following APIs

- `__call__()`: call the `RagasEvaluatorChain` directly on the result of a QA chain.
- `evaluate()`: evaluate on a list of examples (with the input queries) and predictions (outputs from the QA chain). 
- `evaluate_run()`: method implemented that is called by langsmith evaluators to evaluate langsmith datasets.

lets see each of them in action to learn more.

In [47]:
result = qa_chain.invoke({"query": eval_questions[1]})
result["result"]

'Queens is the borough of New York City with the highest population.'

In [48]:
key_mapping = {
    "query": "question",
    "result": "answer",
    "source_documents": "contexts"
}

result_updated = {}
for old_key, new_key in key_mapping.items():
    if old_key in result:
        result_updated[new_key] = result[old_key]


In [49]:
result_updated

{'question': 'Which borough of New York City has the highest population?',
 'answer': 'Queens is the borough of New York City with the highest population.',
 'contexts': [Document(id='841396be-e698-4cab-9ac3-00cc46b2f6db', metadata={'source': './nyc_text.txt', 'row': 33}, page_content="New York: Manhattan Island is loosely divided into the Lower\noften called New York City or NYC: Midtown\nis the most populous city in the United States. With a 2020 population of 8: and Uptown regions. Uptown Manhattan is divided by Central Park into the Upper East Side and the Upper West Side\n804: and above the park is Harlem\n190 distributed over 300.46 square miles (778.2 km2): bordering the Bronx (Bronx County).\nNew York City is the most densely populated major city in the United States and more than twice as populous as Los Angeles: None\nthe nation's second-largest city. New York City is located at the southern tip of New York State. It constitutes the geographical and demographic center of both

In [50]:
pip install --no-cache-dir recordclass

     ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
     ---------------------------------------- 1.3/1.3 MB 10.7 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for recordclass: filename=recordclass-0.22.1-cp313-cp313-win_amd64.whl size=135477 sha256=d50bac033d4da342147ebc88a31a255ab4e9fe5cf7d66e5bc6928385aa52fd76
  Stored in directory: C:\Users\alvar\AppData\Local\Temp\pip-ephem-wheel-cache-azq5vwar\wheels\27\a2\67\8a2fc4fa24702b5000106799cc999151f050e7efc31f310957
Successfully built recordclass
Note: you may need to restart the kernel to use updated packages.


In [51]:
pip install ragas==0.1.9

In [52]:
from ragas.integrations.langchain import EvaluatorChain 
# from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_relevancy,
    context_recall,
)

# create evaluation chains
faithfulness_chain   = EvaluatorChain(metric=faithfulness)
answer_rel_chain     = EvaluatorChain(metric=answer_relevancy)
context_rel_chain    = EvaluatorChain(metric=context_relevancy)
context_recall_chain = EvaluatorChain(metric=context_recall)

c:\Users\alvar\AppData\Local\Programs\Python\Python313\Lib\site-packages\ragas\metrics\__init__.py:1: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from ragas.metrics._answer_correctness import AnswerCorrectness, answer_correctness
c:\Users\alvar\AppData\Local\Programs\Python\Python313\Lib\site-packages\ragas\metrics\__init__.py:4: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain.pydantic_v1 module was a compatibility shim for pydanti

1. `__call__()`

Directly run the evaluation chain with the results from the QA chain. Do note that metrics like context_relevancy and faithfulness require the `source_documents` to be present.

In [53]:
# Recheck the result that we are going to validate.
result

{'query': 'Which borough of New York City has the highest population?',
 'result': 'Queens is the borough of New York City with the highest population.',
 'source_documents': [Document(id='841396be-e698-4cab-9ac3-00cc46b2f6db', metadata={'source': './nyc_text.txt', 'row': 33}, page_content="New York: Manhattan Island is loosely divided into the Lower\noften called New York City or NYC: Midtown\nis the most populous city in the United States. With a 2020 population of 8: and Uptown regions. Uptown Manhattan is divided by Central Park into the Upper East Side and the Upper West Side\n804: and above the park is Harlem\n190 distributed over 300.46 square miles (778.2 km2): bordering the Bronx (Bronx County).\nNew York City is the most densely populated major city in the United States and more than twice as populous as Los Angeles: None\nthe nation's second-largest city. New York City is located at the southern tip of New York State. It constitutes the geographical and demographic center of

**Faithfulness**

In [54]:
eval_result = faithfulness_chain(result_updated)
eval_result["faithfulness_score"]

C:\Users\alvar\AppData\Local\Temp\ipykernel_17568\1823623380.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  eval_result = faithfulness_chain(result_updated)


RuntimeError: asyncio.run() cannot be called from a running event loop

High faithfulness_score means that there are exact consistency between the source documents and the answer.

You can check lower faithfulness scores by changing the result (answer from LLM) or source_documents to something else.

In [ ]:
fake_result = result.copy()
fake_result["result"] = "we are the champions"
eval_result = faithfulness_chain(fake_result)
eval_result["faithfulness_score"]

**Context Relevancy**

In [ ]:
eval_result = context_recall_chain(result)
eval_result["context_recall_score"]

High context_recall_score means that the ground truth is present in the source documents.

You can check lower context recall scores by changing the source_documents to something else.

In [ ]:
from langchain.schema import Document
fake_result = result.copy()
fake_result["source_documents"] = [Document(page_content="I love christmas")]
eval_result = context_recall_chain(fake_result)
eval_result["context_recall_score"]

2. `evaluate()`

Evaluate a list of inputs/queries and the outputs/predictions from the QA chain.

In [ ]:
# run the queries as a batch for efficiency
predictions = qa_chain.batch(examples)

# evaluate
print("evaluating...")
r = faithfulness_chain.evaluate(examples, predictions)
r

In [ ]:
# evaluate context recall
print("evaluating...")
r = context_recall_chain.evaluate(examples, predictions)
r